In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [3]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

top_7_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(7)

In [4]:
top_7_products.head(7)

,article_id,customer_id
7274,751471001,1595
25078,918522001,1516
24268,909370001,1514
24855,915526001,1508
25055,918292001,1503
24858,915529003,1485
25467,924243001,1484


In [5]:
top_7_products.shape

(7, 2)

In [6]:
transactions_train['repeat_purchase'] = transactions_train.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

In [7]:
filler_products = transactions_train[(transactions_train['repeat_purchase'] > 1) & (transactions_train['t_year'] == 2020)].\
groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(5)

In [8]:
top_12_products = pd.concat([top_7_products, filler_products], axis = 0)[['article_id']].drop_duplicates()

In [9]:
top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [10]:
top_12_products

,article_id
7274,0751471001
25078,0918522001
24268,0909370001
24855,0915526001
25055,0918292001
24858,0915529003
25467,0924243001
6692,0706016001
6693,0706016002
406,0372860001


In [11]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)